# Cassandra with Python

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.  

## Let's work the data from the event_datafile_new.csv file, located in the workspace directory. The event_datafile_new.csv contains the following columns:

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of how the denormalized data looks at the end of these processes after executing the above code:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    # To establish connection and begin executing queries, need a session
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# reader file

In [8]:
df_table = pd.read_csv('./event_datafile_new.csv')

In [9]:
# drop table
query_drop_table_artist_song_length = "DROP TABLE IF EXISTS table_artist_song_length"
try:
    session.execute(query_drop_table_artist_song_length)
except Exception as e:
    print(e)

### Creating the table_artist_song_length table.

In [10]:
query_1 = "CREATE TABLE IF NOT EXISTS table_artist_song_length"
query_1 = query_1 + "(sessionId int,\
                      itemInSession int,\
                      artist text,\
                      song text,\
                      length float,\
                      PRIMARY KEY(sessionId, itemInSession)\
                      )"

try:
    session.execute(query_1)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query_1 = "INSERT INTO table_artist_song_length (sessionId, itemInSession, artist, song, length)"
        query_1 = query_1 + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query_1, (
            int(line[8]),
            int(line[3]),
            str(line[0]),
            str(line[9]),
            float(line[5])
        ))

In [12]:
## SELECT statement to verify the data was entered into the table

query_select_table_artist_song_length = "select artist,\
                                         song,\
                                         length\
                                         from table_artist_song_length\
                                         WHERE sessionId = 338 AND itemInSession = 4"                    

try:
    rows = session.execute(query_select_table_artist_song_length)
except Exception as e:
    print(e)
    
print("Select 1: ")    
for row in rows:
    print (row)

Select 1: 
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
# drop table
query_drop_table_artist_song = "DROP TABLE IF EXISTS table_artist_song"

try:
    session.execute(query_drop_table_artist_song)
except Exception as e:
    print(e)  

### Creating the table_artist_song table.

In [14]:
query = "CREATE TABLE IF NOT EXISTS table_artist_song"
query = query + "(userId int,\
                  sessionId int,\
                  itemInSession int,\
                  artist text,\
                  song text,\
                  firstName text,\
                  lastName text,\
                  PRIMARY KEY((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession DESC)"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query_2 = "INSERT INTO table_artist_song (sessionId, itemInSession, artist, song, userId, firstName, lastName)"
        query_2 = query_2 + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query_2, (
            int(line[8]),
            int(line[3]),
            str(line[0]),
            str(line[9]),
            int(line[10]), 
            str(line[1]),
            str(line[4])
        ))                   

In [16]:
query_select_table_artist_song = "select artist, song, firstName, lastName from table_artist_song WHERE userId = 10 AND sessionId = 182"  

try:
    rows = session.execute(query_select_table_artist_song)
except Exception as e:
    print(e)
    
print("Select 2: ")    
for row in rows:
    print (row)   

Select 2: 
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


In [17]:
query_drop_table_fistName_lastName = "DROP TABLE IF EXISTS table_fistName_lastName"

try:
    session.execute(query_drop_table_fistName_lastName)
except Exception as e:
    print(e)  

### Creating the table_artist_song table.

In [18]:
query = "CREATE TABLE IF NOT EXISTS table_fistName_lastName"
query = query + "(song text,\
                  userId int,\
                  firstName text,\
                  lastName text,\
                  PRIMARY KEY(song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)


In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query_2 = "INSERT INTO table_fistName_lastName (song, userId, firstName, lastName)"
        query_2 = query_2 + "VALUES(%s, %s, %s, %s)"
        session.execute(query_2, (
            str(line[9]),
            int(line[10]),
            str(line[1]),
            str(line[4])
        ))

### Creating the table_artist_song table.

In [20]:
query_select_table_fistName_lastName = "select firstName, lastName from table_fistName_lastName WHERE song = 'All Hands Against His Own'"  

try:
    rows = session.execute(query_select_table_fistName_lastName)
except Exception as e:
    print(e)
    
print("Select 3: ")    
for row in rows:
    print (row)  
                

Select 3: 
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions

try:
    session.execute(query_drop_table_artist_song_length)
except Exception as e:
    print(e)

try:
    session.execute(query_drop_table_artist_song)
except Exception as e:
    print(e)    
    

try:
    session.execute(query_drop_table_fistName_lastName)
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()